# David Edwards
## Coursera Applied Data Science Capstone

## Introduction/Business Problem

Cheba Hut (https://chebahut.com/) is a Cannabis-themed sandwich shop based in my hometown of Fort Collins, Colorado.  They are a franchise with approximately 36 locations nationwide.  They are always looking to expand their market, and have a good sense of what makes their franchises work.  The primary indicators of success for them are:
1. Proximity to College/University
2. Lack of alternate restaurant locations
3. Local cannabis laws

I propose performing a "Neighborhood" search that, instead of concentrating on neighborhoods within a city, will concentrate on neighborhoods around colleges/universities in the US.  I propose using the following criteria for determining similarity between Universities:
1. Food (or sandwich shops) per enrolled student
2. laxity of cannibas laws
3. existing Cheba Hut Locations

### Data Sources

#### Marijuana Laws By State
https://data.world/sya/marijuana-laws-by-state
#### College and University Campuses
https://hifld-geoplatform.opendata.arcgis.com/datasets/colleges-and-universities-campuses
#### Cheba Hut Locations
https://chebahut.com/locations

In [54]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [21]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


In [52]:
CLIENT_ID = "UYBKEUMP4GTGASTSES4JECTXW5GKVHSSXRGWD1OKTZUDDDRJ"
CLIENT_SECRET = "Q2FJQLMA1ZTOPDVGTI5BGL5ALLASED24TPOYYFFAUJFOV1O1"
VERSION='20180323'


In [41]:
df = pd.read_csv("Colleges_and_Universities_Campuses.csv")
df.shape

(6005, 27)

In [42]:
df.head()

,OBJECTID,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,...,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,SHAPE__Area,SHAPE__Length
0,1,45821004,WEST COAST UNIVERSITY - ONTARIO CAMPUS,"2855 E. GUASTI RD. ONTARIO, CA 91761",ONTARIO,CA,91761,NOT AVAILABLE,(909) 467-6100,3,...,https://westcoastuniversity.edu/campuses/ontar...,1535068800000,IMAGERY/OTHER,1550448000000,https://westcoastuniversity.edu/campuses/ontar...,-999,-999,NOT AVAILABLE,6893.691406,360.534365
1,2,36639501,SUFFOLK COUNTY COMMUNITY COLLEGE - MICHAEL J. ...,CROOKED HILL ROAD,BRENTWOOD,NY,11717,NOT AVAILABLE,NOT AVAILABLE,1,...,https://www.sunysuffolk.edu/about-suffolk/camp...,1554768000000,IMAGERY/OTHER,1554768000000,http://www3.sunysuffolk.edu/About/CampusInfo.asp,9000,-999,NOT AVAILABLE,831833.769531,3408.024412
2,3,45821003,WEST COAST UNIVERSITY - LA PALMA COUNTY CAMPUS,2411 W. LA PALMA AVE.,ANAHEIM,CA,92801,NOT AVAILABLE,(714) 876-6082,3,...,https://westcoastuniversity.edu/academics/libr...,1550448000000,IMAGERY/OTHER,1550448000000,https://westcoastuniversity.edu/academics/libr...,-999,-999,NOT AVAILABLE,6032.871094,310.813123
3,4,45821002,WEST COAST UNIVERSITY - MIAMI CAMPUS,9250 NW 36TH STREET,DORAL,FL,33178,NOT AVAILABLE,(786) 501-7052,3,...,https://westcoastuniversity.edu/campuses/miami...,1535500800000,IMAGERY/OTHER,1550448000000,https://westcoastuniversity.edu/academics/libr...,-999,-999,NOT AVAILABLE,18280.937500,540.831186
4,5,45821001,WEST COAST UNIVERSITY - LOS ANGELES CENTER FOR...,590 NORTH VERMONT AVENUE,LOS ANGELES,CA,90004,NOT AVAILABLE,(323) 473-5672,3,...,https://westcoastuniversity.edu/campuses/los-a...,1550448000000,IMAGERY/OTHER,1550448000000,https://westcoastuniversity.edu/academics/libr...,-999,-999,NOT AVAILABLE,8609.371094,447.077456


#### We're interested in the names and locations, not so much the rest of this info

In [43]:
df.drop(columns=['OBJECTID', 'UNIQUEID', 'ZIP4', 'TELEPHONE', 'SOURCE', 'SOURCEDATE', 'VAL_DATE', 'WEBSITE', 'COUNTY', 'COUNTYFIPS', 'COUNTRY', 'NAICS_CODE', 'NAICS_DESC', 'VAL_METHOD', 'SHELTERID', 'SHAPE__Area', 'SHAPE__Length'], inplace=True)

I work at Colorado State University, so I wanted to see the pertinent data for my location, and to get a better idea of what the TOT_ENROLL and POPULATION means.  Seeing a TOT_ENROLL of 33083 and POPULATION of 40766 tells me that the former is the number of students, and the POPULATION is students+employees.  I'll use total POPULATION as employees need to eat sandwiches also.

In [44]:
df[(df['TOT_ENROLL'] != -999) & (df['STATE']=='CO')].sort_values('TOT_ENROLL', ascending=False)

,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,POPULATION,TOT_ENROLL,TOT_EMP
2973,UNIVERSITY OF COLORADO BOULDER,REGENT DRIVE AT BROADWAY,BOULDER,CO,80309,1,A,44498,35338,9160
2979,COLORADO STATE UNIVERSITY-FORT COLLINS,102 ADMINISTRATION BUILDING,FORT COLLINS,CO,80523,1,A,40766,33083,7683
2980,COLORADO TECHNICAL UNIVERSITY-COLORADO SPRINGS,4435 N CHESTNUT STREET,COLORADO SPRINGS,CO,80907,3,A,27508,25517,1991
2971,UNIVERSITY OF COLORADO DENVER/ANSCHUTZ MEDICAL...,"1380 LAWRENCE STREET, LAWRENCE STREET CENTER, ...",DENVER,CO,80217,1,A,36608,24839,11769
5487,METROPOLITAN STATE UNIVERSITY OF DENVER,SPEER BLVD AND COLFAX AVE,DENVER,CO,80217,1,A,22515,20304,2211
...,...,...,...,...,...,...,...,...,...,...
2255,NATIONAL AMERICAN UNIVERSITY-COLORADO SPRINGS ...,"1079 SPACE CENTER DRIVE, SUITE 140",COLORADO SPRINGS,CO,80915,3,A,171,152,19
2652,COLORADO MEDIA SCHOOL,404 SOUTH UPHAM ST.,LAKEWOOD,CO,80226,3,A,149,123,26
2759,COLORADO ACADEMY OF VETERINARY TECHNOLOGY,2766 JANITELL ROAD,COLORADO SPRINGS,CO,80906,3,A,127,109,18
2426,THE SALON PROFESSIONAL ACADEMY-GRAND JUNCTION,432 NORTH AVENUE,GRAND JUNCTION,CO,81501,3,A,76,62,14


### Now I know we can get rid of any POPULATION values of -999.  I will also eliminate POPs < 1000
The dataset mentions that means that -999 are unknown, and I want to eliminate very small schools.

In [49]:

df = df[df['POPULATION'] >=1000]
df.shape

(2930, 10)

In [50]:
df

,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,POPULATION,TOT_ENROLL,TOT_EMP
1,SUFFOLK COUNTY COMMUNITY COLLEGE - MICHAEL J. ...,CROOKED HILL ROAD,BRENTWOOD,NY,11717,1,A,9000,9000,-999
121,MIRACOSTA COLLEGE - SAN ELIJO CAMPUS,3333 MANCHESTER AVENUE,ENCINITAS,CA,92007,1,A,4000,4000,-999
130,MADISON AREA TECHNICAL COLLEGE - WEST,302 S GAMMON ROAD,MADISON,WI,53717,1,A,3572,3572,-999
331,SOUTHWEST TENNESSEE COMMUNITY COLLEGE - WHITEH...,"3035 DIRECTORS ROW, BUILDING 6",MEMPHIS,TN,38131,1,A,1198,872,326
415,COLUMBIA STATE COMMUNITY COLLEGE - LAWRENCE CO...,1620 SPRINGER RD,LAWRENCEBURG,TN,38464,1,A,1084,715,369
...,...,...,...,...,...,...,...,...,...,...
6000,SOUTH FLORIDA INSTITUTE OF TECHNOLOGY,"720 NW 27TH AVENUE, 2ND FLOOR",MIAMI,FL,33125,3,A,1401,1308,93
6001,ROANE STATE COMMUNITY COLLEGE,276 PATTON LANE,HARRIMAN,TN,37748,1,A,6210,5626,584
6002,COMMUNITY COLLEGE OF VERMONT,660 ELM STREET,MONTPELIER,VT,05602,1,A,6302,5504,798
6003,NEW RIVER COMMUNITY AND TECHNICAL COLLEGE,280 UNIVERSITY DRIVE,BEAVER,WV,25813,1,A,1323,1172,151


### We're only interested in sandwich shops
Sandwich shop is categoryid = 4bf58dd8d48988d1c5941735

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?categoryid=4bf58dd8d48988d1c5941735&client_id={}&client_secret={}&near={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, '280 UNIVERSITY DRIVE, BEAVER, WV 25813', VERSION, 500, 100)
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']


In [77]:
url = 'https://api.foursquare.com/v2/venues/explore?categoryid=4bf58dd8d48988d1c5941735,4bf58dd8d48988d116941735&client_id={}&client_secret={}&ll={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, '40.572760, -105.086184', VERSION, 5000, 100)
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']


In [80]:
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues[nearby_venues['name'].str.contains('Sub')]

,name,lat,lng
1,Cheba Hut Toasted Subs,40.578200,-105.076669
69,Firehouse Subs,40.612113,-105.075989
